## This notebook will estimate CO2 emissions for a workload on mltgpu

Below is the info from Robert Adesam, notebook by Simon Hengchen.

Run the last cell with your own values.

- CPU: 2x Intel® Xeon® Processor E5-2640 v4 https://ark.intel.com/content/www/us/en/ark/products/92984/intel-xeon-processor-e5-2640-v4-25m-cache-2-40-ghz.html
- GPU: 4x Nvidia GeForce GTX 1080ti https://www.nvidia.com/en-sg/geforce/products/10series/geforce-gtx-1080-ti/
- Motherboard: ASUSTeK Z10PE-D8 WS
- Memory: 8x 32GB ECC DDR4 RDIMM 2133MHz
- Storage: 
    - 2x Seagate Barracuda ST2000DM006 (data disks in raid1)
    - Samsung 960EVO NVMe M.2 (scratch disk for doing GPU compute snapshots) 
    - Samsung 860 pro (system disk)
    - User homes are NFS shares, **TBD**

In [85]:
# TDP in watt per device

TPD_device = {"CPU" : 90,
              "GPU" : 250,
              "RAM" : 5, # guesstimate
              "HDD" : 12.5, # guesstimate of 12.5W per 7200 rpm
              "SSD" : 0.5, # biggest value here https://www.anandtech.com/show/12408/the-samsung-860-evo-m2-2tb-ssd-review/
}

In [86]:
def calculate_draw(CPU=1, GPU=0):
    """
    Will calculate and return TDP in Watts. We consider the RAM to be always utilised.
    """
    RAM = 8
    HDD = 2
    SSD = 1
    
    if GPU > 0:
        SSD = 2 # one of the SSDs is scratch for GPU
    """
    print("CPU", CPU)
    print("GPU", GPU)
    print("RAM", RAM)
    print("HDD", HDD)
    print("SSD", SSD)
    """
    TDP = CPU * TPD_device["CPU"] + \
        GPU * TPD_device["GPU"] + \
        RAM * TPD_device["RAM"] + \
        HDD * TPD_device["HDD"] + \
        SSD * TPD_device["SSD"]
    return TDP

In [87]:
def calculate_co2(TDP, hours):
    """
    Based on data for Sweden from https://www.rensmart.com/Calculators/KWH-to-CO2, will calculate co2 emissions in kg
    """
    kWh = TDP / 1000 * hours
    CO2 =  kWh * 0.013 # (0.013 is the CO2 kg per kWh)
    return CO2

In [88]:
def print_paris(value_done):
    """
    Takes a float between 0 and 1 and outputs a progress bar
    """
    print("☠️☠️☠️ [{0:80s}] {1:.1f}% ☠️☠️☠️".format('☁️' * int(value_done * 50), value_done * 100))

In [89]:
def co2_to_bananas(CO2):
    """
    Will convert CO2 emissions in kg to Netflix usage, banana usage, avocado usage, etc.
    ## DISCLAIMER: I have not checked in details whether the "co2 to .." figures are absolutely correct (for Sweden or elsewhere)
    """
    paris_agreement = CO2/2000  # every human should spend around 2 tonnes a year max; here i consider tonne as metric tonne https://www.vox.com/22291568/climate-change-carbon-footprint-greta-thunberg-un-emissions-gap-report
    bananas = CO2/0.08 # 1 banana is 80g https://www.theguardian.com/environment/green-living-blog/2010/jul/01/carbon-footprint-banana
    netflix_hour = CO2/0.036 # 1 hour of netflix is 36g https://www.carbonbrief.org/factcheck-what-is-the-carbon-footprint-of-streaming-video-on-netflix
    km_driven = CO2/0.185 # 5km = 0.925 https://www.carbonbrief.org/factcheck-what-is-the-carbon-footprint-of-streaming-video-on-netflix
    avocados = CO2/0.423 # 2 avocados from Mexico = 846.36g https://www.weforum.org/agenda/2020/02/avocado-environment-cost-food-mexico/
    
    print("CO2 usage is estimated to be", str(round(CO2,3))+" kg", "and equivalent to:")
    print("===================")
    print(round(bananas,3), "bananas")
    print(round(netflix_hour,3), "hours on Netflix if your TV doesn't consume any electricity")
    print(round(km_driven,3), "kilometers driven")
    print(round(avocados,3), "avocados from Mexico")
    print()
    print(round(paris_agreement*100,3), "% of your yearly Paris Agreement share")
    print_paris(paris_agreement)

In [91]:
## Change values in calculate_draw() (#CPU, #GPU) and in calculate_co2 (# of hours)

TDP = calculate_draw(2,4)
CO2 = calculate_co2(TDP, 1600)
co2_to_bananas(CO2)

CO2 usage is estimated to be 25.917 kg and equivalent to:
323.96 bananas
719.911 hours on Netflix if your TV doesn't consume any electricity
140.091 kilometers driven
61.269 avocados from Mexico

1.296 % of your yearly Paris Agreement share
☠️☠️☠️ [                                                                                ] 1.3% ☠️☠️☠️
